In [21]:
import copy
import tequila as tq
from tequila import gates
tq.show_available_simulators()

backend         | wfn        | sampling   | noise      | installed 
--------------------------------------------------------------------
qulacs_gpu      | False      | False      | False      | False     
qulacs          | False      | False      | False      | False     
qibo            | False      | False      | False      | False     
qiskit          | True       | True       | True       | True      
cirq            | True       | True       | True       | True      
pyquil          | True       | True       | True       | True      
symbolic        | True       | False      | False      | True      
qlm             | False      | False      | False      | False     


In [18]:
def xor(q1: int, q2: int, output: int):
    circuit = tq.QCircuit()
    circuit += gates.CNOT(q1, output)
    circuit += gates.CNOT(q2, output)
    return circuit

In [19]:
def sudoku_oracle(inputs: list[int], tmp: list[int], output: int):
    circuit = tq.QCircuit()

    circuit += xor(inputs[0], inputs[1], tmp[0])
    circuit += xor(inputs[1], inputs[3], tmp[1])
    circuit += xor(inputs[0], inputs[2], tmp[2])
    circuit += xor(inputs[2], inputs[3], tmp[3])

    circuit_t = copy.deepcopy(circuit)

    circuit += gates.X(output, tmp)

    circuit += circuit_t

    return circuit

In [25]:
def reflect_zero(inputs: list[int]):
    circuit = tq.QCircuit()

    circuit += gates.X(inputs)
    circuit += gates.Z(inputs[-1], inputs[:len(inputs) - 1])
    circuit += gates.X(inputs)

    return circuit

In [26]:
def grover_circuit():
    inputs = [i for i in range(4)]
    tmp = [i + 4 for i in range(4)]
    pkb = 8

    circuit = tq.QCircuit()

    circuit += gates.H(inputs)
    circuit += gates.X(pkb)
    circuit += gates.H(pkb)

    for _ in range(2):
        circuit += sudoku_oracle(inputs, tmp, pkb)

        circuit += gates.H(inputs)
        circuit += reflect_zero(inputs)
        circuit += gates.H(inputs)

    return circuit

In [27]:
measurements = tq.simulate(grover_circuit(), samples=1000, read_out_qubits=[0,1,2,3])
print(measurements)

+2.0000|0001> +454.0000|0110> +4.0000|1011> +9.0000|0011> +5.0000|0101> +2.0000|1110> +5.0000|1100> +5.0000|1101> +3.0000|1111> +6.0000|1000> +496.0000|1001> +3.0000|1010> +2.0000|0100> +2.0000|0000> +2.0000|0010> 
